# Linear model try
V1.0  
This version is a first build-up of the whole system. 

### Import Libs

In [1]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import os
import numpy as np
import matplotlib.pyplot as plt

from model_dataset import HandshapeDataset, HandshapeDict, HandshapeIndexor, DS_Tools
from paths import *
from model_model import HandshapePredictor
from model_configs import *
from utils import *
from recorder import *
from graph_tools import GraphTool, Plotter, Smoother

### Init Model

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
model = HandshapePredictor(
    input_dim=in_dim, 
    enc_lat_dims=enc_lat_dims, 
    hid_dim=hid_dim, 
    dec_lat_dims=dec_lat_dims, 
    output_dim=out_dim
)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [3]:
model

HandshapePredictor(
  (encoder): Sequential(
    (0): LinPack(
      (lin): Linear(in_features=63, out_features=128, bias=True)
      (relu): ReLU()
    )
    (1): ResBlock(
      (lin1): Linear(in_features=128, out_features=128, bias=True)
      (lin2): Linear(in_features=128, out_features=128, bias=True)
      (relu): ReLU()
    )
    (2): LinPack(
      (lin): Linear(in_features=128, out_features=32, bias=True)
      (relu): ReLU()
    )
    (3): ResBlock(
      (lin1): Linear(in_features=32, out_features=32, bias=True)
      (lin2): Linear(in_features=32, out_features=32, bias=True)
      (relu): ReLU()
    )
    (4): Linear(in_features=32, out_features=8, bias=True)
  )
  (decoder): Sequential(
    (0): LinPack(
      (lin): Linear(in_features=8, out_features=16, bias=True)
      (relu): ReLU()
    )
    (1): LinPack(
      (lin): Linear(in_features=16, out_features=64, bias=True)
      (relu): ReLU()
    )
    (2): Linear(in_features=64, out_features=93, bias=True)
  )
)

In [4]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

54997


In [5]:
# READ = False
READ = True

In [6]:
if READ: 
    ts = "0920183935"
    stop_epoch = "799"
    model_raw_name = "CP_{}_{}".format(ts, stop_epoch)
    model_name = model_raw_name + ".pt"
    # model_name = "PT_0816184446_1349_full.pt"
    model_path = os.path.join(model_save_dir, model_name)
    state = torch.load(model_path)
    model.load_state_dict(state)
    model.to(device)

In [7]:
css_in = """
<head>
    <link rel="stylesheet" href="{}">
</head>
""".format(outstyle_path)

control_script_pre = """
<script>
const integerInput = document.getElementById('integerInput');
const outputDiv = document.getElementById('outputDiv');

const stringsList ="""

control_script_post = """
integerInput.addEventListener('input', () => {
  const selectedIndex = parseInt(integerInput.value);
  
  if (selectedIndex >= 0 && selectedIndex < stringsList.length) {
    const selectedString = stringsList[selectedIndex];
    outputDiv.textContent = selectedString;
  } else {
    outputDiv.textContent = "無";
  }
});
</script>
"""


frame_pred_vis = """
<div class="container">
  <input type="number" id="integerInput" min="0">
  <div class="output" id="outputDiv"></div>
</div>
"""

In [8]:
data_path = os.path.join(data_dir, data_name + "_data.npz")
tag_path = os.path.join(data_dir, data_name + "_tag.npz")

In [10]:
hsdict = HandshapeDict(tag_path)

for vd in os.listdir(det_dir): 
    for clip in os.listdir(det_dir + vd + "/")[876:]: 
        print(clip)
        for whether_smooth in ["non", "ma"]: 
            smooth_dir = os.path.join(spec_dir, whether_smooth + "/")
            pic_smooth_dir = os.path.join(spec_pic_dir, whether_smooth + "/")
            mk(smooth_dir)
            mk(pic_smooth_dir)

            for side in ["Right", "Left"]:
                find_name = "{}_{}".format(side, clip)
                reverse_find_name = "{}_{}".format(clip, side)

                gt = GraphTool(graph_dir, find_name)
                gt.interpolate(window_size=2)

                if whether_smooth == "non": 
                    smoothed_features = gt.interpolated_features
                elif whether_smooth == "ma": 
                    smoothed_features = Smoother.moving_average(gt.interpolated_features)
                else: 
                    smoothed_features = gt.interpolated_features

                this_features = torch.from_numpy(smoothed_features.copy())
                batch_num, lm_num, dim_num = this_features.size()

                x = this_features
                x = x.to(device)
                x = x.to(torch.float32)

                hid_rep, pred = model.predict(x, hsdict)

                hid_rep = hid_rep.cpu().detach().numpy()

                html = """"""

                html += css_in

                html += """<h1>{}</h1><br>""".format(reverse_find_name)

                html += frame_pred_vis

                html += Plotter.plot_spectrogram(
                    hid_rep, 
                    title="ML Spectrogram" + side, 
                    save_path= os.path.join(pic_smooth_dir, reverse_find_name + "_mlspec")
                )

                html += Plotter.plot_line_graph(hid_rep, ["0", "1", "2", "3", "4"], 
                                                "ML Linegraph" + side, y_axis_label="Val", 
                                                save_path= os.path.join(pic_smooth_dir, reverse_find_name + "_mlline"))
                
                html += control_script_pre + str(pred) + control_script_post
                
                Plotter.write_to_html(html, "{}{}.html".format(smooth_dir, reverse_find_name))

C_03_037-PROTECTION-0JUT-416
HKSL_lesson_only91-GRASS-10Q9-554
HKSL_lesson_only427-NORTH-0KON-903
B_03_016-VACATION-0K27-170
B_03_058-FIGHT-0OIJ-206
HKSL_lesson_only287-WATERMELON-12BV-761
C_02_046-BALANCE-0NJJ-354
C_03_020-SQUIRREL-0PRU-404
HKSL_lesson_only474-RECOVER-0NLN-952
C_01_071-STAFF-103N-298
HKSL_lesson_only423-OUTSIDE-0M8M-899
HKSL_lesson_only445-INJECTION_1-0OIJ-922
C_02_056-BENEFIT-0MBT-364
HKSL_lesson_only394-BUT-0JQ6-870
C_01_036-^HARD_TEXTURE-0U3C-268
B_02_077-PILE-0M06-140
HKSL_lesson_only425-SOUTH-0KQN-901
C_01_015-MOCK-0O35-244
B_03_051-CHALLENGE-0OOH-201
B_03_018-IMAGINE-0O7J-172
B_02_059-^ACCIDENT_3-0O8F-123
C_02_058-GULLIBLE-0MTP-366
HKSL_lesson_only79-WALK-13BG-542
HKSL_lesson_only66-HIKE-122C-529
B_02_076-WONDER-12MN-139
B_01_077-FATHER-0SHO-64
C_01_008-TALK-12OR-238
HKSL_lesson_only489-ASKING_FOR_TROUBLE-10FA-969
HKSL_lesson_only8-AGAIN-0KCD-466
HKSL_lesson_only405-DATE-0V84-881
HKSL_lesson_only179-SOCIAL_WORKER-0U9U-646
C_01_030-WASTE-0RBA-262
C_01_031-LETTER-